In [1]:
import sys
import os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import flopy

flopy is installed in C:\Users\emmal\Anaconda3\lib\site-packages\flopy


In [7]:
mpexe = "../../exe/mp7.exe"
mfexe = "../../exe/mf6.exe"
model_dir="working"
model_name="test_modpath"

In [18]:
nper, nstp, perlen, tsmult = 1, 1, 1., 1.
nlay, nrow, ncol = 3, 21, 20
delr = delc = 500.
top = 400.
botm = [220., 200., 0.]
laytyp = [1, 0, 0]
kh = [50., 0.01, 200.]
kv = [10., 0.01, 20.]

wel_loc = (2, 10, 9)
wel_q = -150000.

rch = 0.005
riv_h = 320.
riv_z = 317.
riv_c = 1.e5

In [19]:
def get_nodes(locs):
    nodes = []
    for k, i, j in locs:
        nodes.append(k * nrow * ncol + i * ncol + j)
    return nodes

In [21]:
# Create the Flopy simulation object
sim = flopy.mf6.MFSimulation(sim_name=model_name, exe_name=mfexe,
                             version='mf6', sim_ws=model_dir)

# Create the Flopy temporal discretization object
periodd = (perlen, nstp, tsmult)
tdis = flopy.mf6.modflow.mftdis.ModflowTdis(sim, pname='tdis',
                                            time_units='DAYS', nper=nper,
                                            perioddata=[periodd])

model_nam_file = '{}.nam'.format(model_name)
gwf = flopy.mf6.ModflowGwf(sim, modelname=model_name,
                           model_nam_file=model_nam_file, save_flows=True)

# Create the Flopy iterative model solver (ims) Package object
ims = flopy.mf6.modflow.mfims.ModflowIms(sim, pname='ims', 
                                         complexity='SIMPLE')


# create gwf file
dis = flopy.mf6.modflow.mfgwfdis.ModflowGwfdis(gwf, pname='dis', nlay=nlay,
                                               nrow=nrow, ncol=ncol,
                                               delr=delr, delc=delc,
                                               top=top,
                                               botm=botm)

# Create the initial conditions package
ic = flopy.mf6.modflow.mfgwfic.ModflowGwfic(gwf, pname='ic', strt=top)

# Create the node property flow package
npf = flopy.mf6.modflow.mfgwfnpf.ModflowGwfnpf(gwf, pname='npf',
                                               icelltype=laytyp, k=kh,
                                               k33=kv)

# recharge
flopy.mf6.modflow.mfgwfrcha.ModflowGwfrcha(gwf, recharge=rch)

# wel
wd = [(wel_loc, wel_q)]
wel=flopy.mf6.modflow.mfgwfwel.ModflowGwfwel(gwf, maxbound=1,
                                         stress_period_data={0: wd})

# river
rd = []
for i in range(nrow):
    rd.append([(0, i, ncol - 1), riv_h, riv_c, riv_z])
riv = flopy.mf6.modflow.mfgwfriv.ModflowGwfriv(gwf, maxbound=len(rd),stress_period_data={0: rd})

# Create the output control package
headfile = '{}.hds'.format(model_name)
head_record = [headfile]
budgetfile = '{}.cbc'.format(model_name)
budget_record = [budgetfile]
saverecord = [('HEAD', 'ALL'),
              ('BUDGET', 'ALL')]
oc = flopy.mf6.modflow.mfgwfoc.ModflowGwfoc(gwf, pname='oc',
                                            saverecord=saverecord,
                                            head_filerecord=head_record,
                                            budget_filerecord=budget_record)

# Write the datasets
sim.write_simulation()
# Run the simulation
success, buff = sim.run_simulation()

writing simulation...
  writing simulation name file...
  writing simulation tdis package...
  writing ims package ims...
  writing model test_modpath...
    writing model name file...
    writing package dis...
    writing package ic...
    writing package npf...
    writing package rcha...
    writing package wel_0...
    writing package riv_0...
    writing package oc...
FloPy is using the following  executable to run the model: ../../exe/mf6.exe
                                   MODFLOW 6
                U.S. GEOLOGICAL SURVEY MODULAR HYDROLOGIC MODEL
                            VERSION 6.0.4 03/13/2019

   MODFLOW 6 compiled Mar 21 2019 15:37:31 with IFORT compiler (ver. 19.0.0)

This software has been approved for release by the U.S. Geological 
Survey (USGS). Although the software has been subjected to rigorous 
review, the USGS reserves the right to update the software as needed 
pursuant to further analysis and review. No warranty, expressed or 
implied, is made by the USGS o

In [25]:
nodew = get_nodes([wel_loc])
cellids = gwf.riv.stress_period_data.get_data()[0]['cellid']
nodesr = get_nodes(cellids)

In [28]:
##forward 

# create modpath files
mpnamf = model_name + '_mp_forward'

# create basic forward tracking modpath simulation
mp = flopy.modpath.Modpath7.create_mp7(modelname=mpnamf, trackdir='forward', flowmodel=gwf, model_ws=model_dir, 
                                       rowcelldivisions=1, columncelldivisions=1, layercelldivisions=1,
                                       exe_name=mpexe)

# write modpath datasets
mp.write_input()

# run modpath
mp.run_model()

FloPy is using the following  executable to run the model: ../../exe/mp7.exe

MODPATH Version 7.2.001   
Program compiled Mar 21 2019 15:41:25 with IFORT compiler (ver. 19.0.0)         
 
 
Run particle tracking simulation ...
Processing Time Step     1 Period     1.  Time =  1.00000E+00  Steady-state flow                                                    

Particle Summary:
         0 particles are pending release.
         0 particles remain active.
         0 particles terminated at boundary faces.
         0 particles terminated at weak sink cells.
         0 particles terminated at weak source cells.
      1260 particles terminated at strong source/sink cells.
         0 particles terminated in cells with a specified zone number.
         0 particles were stranded in inactive or dry cells.
         0 particles were unreleased.
         0 particles have an unknown status.
 
Normal termination.                                                        


(True, [])